In [2]:
import re
from io import StringIO
import argparse
import os.path as op
import os
import subprocess
import gzip

import glob
import pandas as pd
import shutil

In [22]:
# fromeyetracking computer 
pati = '/mnt_03/ds-dnumrisk/sourcedata/eyetracking_risk_renamed' # tryouts'

file = 'sub-01' # '16_26_17'
fn = op.join(pati, f'{file}.edf')


In [9]:
# from experiment_risk folder
pati = '/mnt_03/ds-dnumrisk/sourcedata/experiment_risk/logs' # tryouts'
file = 'sub-01/ses-1/sub-1_ses-1_task-risk_non-symbolic'

pati = '/mnt_03/ds-dnumrisk/sourcedata/' # tryouts'
file = 'ex-file-gilles_sub-02_ses-3t2_task-task_run-1'

fn = op.join(pati, f'{file}.edf')
fn

'/mnt_03/ds-dnumrisk/sourcedata/experiment_risk/logs/sub-01/ses-1/sub-1_ses-1_task-risk_non-symbolic.edf'

In [33]:
# extract_data

asc_fn = fn.replace('.edf', '.asc')
gaze_target_fn = fn.replace('.edf', '.gaz.gz')

edf2acs_cmd = '/home/ubuntu/git/riskeye/edf2asc'

# get gaze
cmd = f'{edf2acs_cmd}  -t -y -z -v -s -vel {fn}'

subprocess.run(cmd, shell=True)

with open(asc_fn, 'rb') as asc_file, gzip.open(gaze_target_fn, 'wb') as target_file:
        target_file.writelines(asc_file)
os.remove(asc_fn)

# get messages
cmd = f'{edf2acs_cmd}   -t -y -z -v -e {fn}'
msg_target_fn = fn.replace('.edf', '.msg.gz')

subprocess.run(cmd, shell=True)

with open(asc_fn, 'rb') as asc_file, gzip.open(msg_target_fn, 'wb') as target_file:
        target_file.writelines(asc_file)
os.remove(asc_fn)


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 3.0 Linux Dec  1 2008 
(c)1995-2007 by SR Research, last modified Dec  1 2008

processing file /mnt_03/ds-dnumrisk/sourcedata/ex-file-gilles_sub-02_ses-3t2_task-task_run-1.edf 
Preamble of file /mnt_03/ds-dnumrisk/sourcedata/ex-file-gilles_sub-02_ses-3t2_task-task_run-1.edf
| DATE: Tue Nov 24 14:35:53 2020                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v4.51 Mar 13 2010                                             |
| CAMERA: EyeLink CL-OL HEAD Version 1.0 Sensor=BI8                           |
| SERIAL NUMBER: CLO-ZAA36                                                    |
| CAMERA_CONFIG: ZAA36140.SCD                                                 |

Converted

## get_experimental_messages

In [ ]:
#def get_experimental_messages(subject, root_folder='/data'):

msg_fn=op.join(pati, f'{file}.msg.gz')   # op.join(pati, f'{file}.edf')

with gzip.open(msg_fn, 'rt') as mfd:
    message_string = mfd.read()

pattern = re.compile(r'MSG\t(?P<timestamp>[0-9]+)\tstart_type-(?P<start_type>.+?)_phase-(?P<phase>[0-9]+)(?:_key-(?P<key>[a-zA-Z]+))?(?:_time-(?P<time>[0-9.]+))?')
# re.compile('MSG\t(?P<timestamp>[0-9.]+)\t(?P<message>[0-9]+_.+)')
message_strings = re.findall(pattern, message_string)
r = re.compile('([0-9]+)_(.+)')

tmp = pd.DataFrame(message_strings, columns=['timestamp', 'message',]).astype({'timestamp':int})#.set_index('timestamp').head(10)
tmp['trial'] = tmp['message'].map(lambda x: r.match(x).group(1)).astype(int)
tmp['type'] = tmp['message'].map(lambda x: r.match(x).group(2)).map(lambda x: 'response' if x.startswith('R') else x)
tmp.head()


In [ ]:
# Compile the regex pattern
pattern = re.compile(r'MSG\t(?P<timestamp>[0-9]+)\tstart_type-(?P<start_type>.+?)_phase-(?P<phase>[0-9]+)(?:_key-(?P<key>[a-zA-Z]+))?(?:_time-(?P<time>[0-9.]+))?')

# Find all matches using the regex
message_strings = pattern.findall(message_string)

# Create a DataFrame with the extracted fields
tmp = pd.DataFrame(message_strings, columns=['timestamp', 'message', 'phase', 'key', 'time']).astype({'timestamp': int, 'phase': int})

# Extract the trial (e.g., 'stim_trial' or 'response_trial') and type information
tmp['trial'] = tmp['message'].map(lambda x: re.match(r'(.+)_trial', x).group(1))
tmp['type'] = tmp['message'].map(lambda x: 'response' if 'response' in x else 'stim')

# Display the first 10 rows
#print(tmp.head(10))

In [ ]:
block = 0
for ix, row in tmp.iterrows():
    if (row.trial == 1) & (row.type == 'rfx'):
        block += 1
    tmp.at[ix, 'block'] = int(block)


tmp['block'] = tmp['block'].astype(int)

tmp = tmp.set_index(['block', 'trial', 'type'], drop=True)

# get_saccades


In [29]:

msg_fn=op.join(pati, f'{file}.msg.gz')   #op.join(dir, f'Rs{subject:02d}rn{run:02d}.msg.gz')

with gzip.open(msg_fn, 'rt') as mfd:
    message_string = mfd.read()
message_string

In [30]:
message_strings = re.findall(re.compile('ESACC\t(?P<info>.+)'), message_string)

csvString = ('\n'.join(message_strings))

s = pd.read_csv(StringIO(csvString), sep='\t', names=['eye', 'start_timestamp', 'end_timestamp', 'duration', 'start_x', 'start_y', 'end_x', 'end_y', 'amp', 'peak_velocity'], na_values=['.', '   .'],)
                    #    dtype={'start_y':float})

s.index.name = 'n'
s.head()



## try functions in prep_eyetrack_data

In [9]:
bids_folder='/mnt_03/ds-dnumrisk'
subject = 1

from prep_eyetrack_data import get_experimental_messages
tmp = get_experimental_messages(subject=subject, bids_folder=bids_folder)

In [4]:
from prep_eyetrack_data import get_saccades
tmp = get_saccades(subject, bids_folder)
tmp.head()

it runs ok


,eye,start_timestamp,end_timestamp,duration,start_x,start_y,end_x,end_y,amp,peak_velocity
n,,,,,,,,,,
0,R,8917630,8917674,46,902.3,438.3,853.8,164.0,5.10,241
1,R,8917838,8917870,34,858.5,159.0,701.4,156.9,3.00,204
2,R,8918232,8918262,32,702.8,154.2,819.1,148.2,2.22,153
3,R,8918400,8918430,32,817.2,144.7,930.9,150.5,2.18,150
4,R,8918692,8918718,28,927.9,150.2,1008.2,147.7,1.54,109


In [7]:
subject

1

In [10]:
import os.path as op

dir = op.join(bids_folder, 'sourcedata/eyetracking_risk_renamed')
fn = op.join(dir, f'sub-{subject:02d}.edf')


In [ ]:
# old code for debugging get_experiment part

for line in message_string.splitlines()[100:110]:  # Print first 10 lines
    print(repr(line))